# Installing reqs

In [ ]:
# pip install pandas scikit-learn matplotlib seaborn wordcloud tqdm PyPDF2 gensim smart-open nltk python-dotenv langchain langchain_community openai langchain_openai chromadb langchain_huggingface pypdf 

# IMPORTS

In [ ]:
import os
import time
import getpass
import chromadb
import openai

from openai import OpenAI
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction
from langchain_openai import ChatOpenAI
from datetime import datetime

# Data Preparation
PATH_DB = './db'
COLLECTION_NAME = 'policy'
DIR_PATH = './documents/policy'

# Chroma vectorstore and Embedding model

Initialize existing persisting storage.

IMPORTANT: make sure to have loaded some documents to the vector database. This can be done by running the ``load_data.py`` in /src.

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
# Instantiate a persistent chroma client in the persist_directory.
# This will automatically load any previously saved collections.
# Learn more at docs.trychroma.com
client_db = chromadb.PersistentClient(path=PATH_DB)

embedder = OllamaEmbeddingFunction(
    model_name="nomic-embed-text",
    url="http://localhost:11434/api/embeddings",
)

# Get the collection.
collection = client_db.get_collection(name=COLLECTION_NAME, embedding_function=embedder)
collection.get()

# Retrieval and generation: Create an agent2agent dialogue pipeline

### The dummy version with meat-eating debate

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY') if 'OPENAI_API_KEY' in os.environ else getpass.getpass()

# Initialize ChatOpenAI instances
devil_agent = ChatOpenAI(api_key=openai.api_key, model="gpt-4o-mini")
angel_agent = ChatOpenAI(api_key=openai.api_key, model="gpt-4o-mini")

# These agents a real chatterboxes, they need some restrains
output_length = 50

# Define system prompts for the two agents
devil_system_prompt = f"""
You are the Devil's Advocate. You will have a debate with the Angel's Advocate. Your mission is to make your case that eating meat is ethically right. 
Always meet your opponent's most recent arguement first and indicate this by writing "Reponse on opponent's arguement: ". 
Then continue by presenting a new argument to streghthen your own point of view, indicate your own view by writing "New aguments made: ".
You have a total of {output_length} words to give your response. Also, start every new sentence with a new row after a row break. 
"""

angel_system_prompt = f"""
You are the Angel's Advocate. You will have a debate with the Devil's Advocate. Your mission is to make the case that eating meat is ethically wrong. 
Always meet your opponent's most recent arguement first and indicate this by writing "\n Reponse on opponent's arguement: ". 
Then continue by presenting a new argument to streghthen your own point of view, indicate your own view by writing "\n New aguments made: ".
You have a total of {output_length} words to give your response. Also, start every new sentence with a new row after a row break. 
"""

# Define initial task prompt for the devil agent
task_prompt = "Discuss how eating meat is ethically right or wrong. You will start by making presenting your point of view on the matter."

# Function to create a debate between the two agents
def run_debate(devil_prompt, angel_prompt, task_prompt, num_rounds=3):
    devil_message = task_prompt
    dialogue = []

    for round_num in range(num_rounds):
        # Devil agent's turn
        devil_response = devil_agent.invoke([{"role": "system", "content": devil_prompt}, {"role": "user", "content": devil_message}])
        devil_message = devil_response.content
        dialogue.append(f"\n \n #####Devil: {devil_message}")
        
        # Angel agent's turn
        angel_response = angel_agent.invoke([{"role": "system", "content": angel_prompt}, {"role": "user", "content": devil_message}])
        angel_message = angel_response.content
        dialogue.append(f"\n \n #####Angel: {angel_message}")
        
        # Prepare for the next round
        devil_message = angel_message

    return dialogue

# Start the timer
start_time = time.time()
# Run the debate
# debate_dialogue = run_debate(devil_system_prompt, angel_system_prompt, task_prompt, num_rounds=2)

# Verbose: Print the full dialogue
# for line in debate_dialogue:
    # print(line)

# End the timer
end_time = time.time()
elapsed_time = end_time - start_time

# Save the dialogue to a .txt file
# current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
# file_name = f"debate_dialogue_{current_time}.txt"
# with open(file_name, "w") as file:
#     for line in debate_dialogue:
#         file.write(line + "\n")
#     file.write(f"\nTime taken: {elapsed_time:.2f} seconds\n")

# print(f"Debate dialogue saved to {file_name}")
# print(f"Time taken: {elapsed_time:.2f} seconds")

# Could be extended by another LLM analysing an long debate script and summarize it. 

# Full version for loopholes [Work in progress]

In [24]:
def build_prompt(message, docs, is_angel=False, output_length=50):
    """Define system prompts for the two agents"""
    
    if not is_angel:
        devil_system_prompt = [
            {
                "role": "system",
                "name": "Devil",
                "content": f"""You are the Devil's Advocate. You will have a debate with the Angel's Advocate.
                    Your task is to exploit loopholes in the university´s governing documents. Be specific and detailed on what you can exploit.
                    You must build your arguments on the information given in the documents you have from the vector storage: {docs}
                    Find any relevant documents and make a point based on the information in the document(s), don't forget to reference the source by stating a quote from the document and the document name.

                    Start with "\n### Devil:\n" followed by your response.
                    Always meet your opponent's most recent arguement first.
                    Then continue by presenting a new argument to streghthen your own point of view.
                    Don't repeat or suggest similar previous arguments.
                    You have a total of {output_length} words to give your response.
                    """
            }, 
            {"role": "user", "content": f"Here's the dialogue: {message}"}
        ]
        return devil_system_prompt
    else:
        angel_system_prompt = [
            {
                "role": "system",
                "name": "Angel",
                "content": f"""
                    You are the Angel's Advocate. You will have a debate with the Devil's Advocate. 
                    Your task is to prevent the exploitation of loopholes in the university´s governing documents.
                    You must build your arguments on the information given in the documents you have from the vector storage: {docs}
                    Find the relevant document for the matter, make a point based on the information given in it and reference the source by stating a quote from the documents and the document name.

                    Start with "\n### Angel:\n" followed by your response.
                    Always meet your opponent's most recent arguement first".
                    Then continue by presenting a new argument to streghthen your own point of view.
                    Don't repeat or suggest similar previous arguments.
                    You have a total of {output_length} words to give your response.
                """
            }, 
            {"role": "user", "content": f"Here's the dialogue: {message}"}
        ]
    return angel_system_prompt

def stream_response(response: str) -> str:
    """Stream response to output."""
    message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            message += (chunk.choices[0].delta.content)
            print(chunk.choices[0].delta.content, end="")
    print("\n\n")
    return message

def get_response(agent: OpenAI, model: str, temperature: float = 0.8, **kwargs) -> str:
    response = agent.chat.completions.create(
        model=model,
        messages=build_prompt(**kwargs),
        stream=True,
        temperature=temperature
    )
    message = stream_response(response)
    return message

def run_debate(task_prompt, num_rounds=3):
    """Function to create a debate between the two agents"""
    
    # Set OpenAI API key
    openai.api_key = os.getenv('OPENAI_API_KEY') if 'OPENAI_API_KEY' in os.environ else getpass.getpass("Enter your OpenAI API key: ")

    devil_agent = OpenAI()
    angel_agent = OpenAI()
    boss_agent = OpenAI()
    
    # MODEL = "gpt-4o-mini"
    MODEL = "gpt-3.5-turbo"
    
    # Query the collection to get the 5 most relevant results
    docs = collection.query(
        query_texts=[task_prompt], n_results=5, include=["documents", "metadatas"]
    )

    if not docs:
        raise ValueError(f"No relevant docs were retrieved!")
        
    message = task_prompt
    dialogue = [message]
    for round_num in range(num_rounds):
        print(f"__________Turn number {round_num+1}__________")
        
        ## Devil agent's turn
        message = get_response(devil_agent, MODEL, message=dialogue, docs=docs, is_angel=False)
        dialogue.append(message)
        
        ## Angel agent's turn
        message = get_response(angel_agent, MODEL, message=dialogue, docs=docs, is_angel=True)
        dialogue.append(message)

        ## Boss agent's take on the arguments.
        boss_response = boss_agent.chat.completions.create(
            model=MODEL,
            messages=[{
                "role": "user", 
                "content": f"""
                    Analyze the following debate arguments {dialogue}.
                    Start with "\n### Boss:\n" followed by your response.
                    If the debate arguments are repetitive and at stand-still, respond with: Case closed.
                """
            }],
            stream=True
        )
        boss_message = stream_response(boss_response)
        if "Case closed" in boss_message:
            print("=> Terminating...")
            dialogue.append(boss_message)
            return "\n\n".join(dialogue)

    return "\n\n".join(dialogue)

## Query it

In [25]:
# Start the timer
start_time = time.time()

# Define initial task prompt for the devil agent
# task_prompt = "Discuss why the policy does allow you to drink beer during working hours. You will start by making presenting your point of view on the matter."
task_prompt = """
    Discuss what is allowed and important to think about for the case of when a Chalmers employee wants to invite a professor with the purpose of recruitment.
    What and how much can Chalmers pay for such as flights, food, drinks?
    Moreover, what do Chalmers offer such as relocation, familiy jobs, etc.
"""

# Run the debate
print("# TASK:", task_prompt)
debate_dialogue = run_debate(task_prompt, num_rounds=4)

# End the timer
end_time = time.time()
elapsed_time = end_time - start_time

# Save the dialogue to a .txt file
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f"out/debate_dialogue_{current_time}.txt"
with open(file_name, "w") as file:
    file.write(debate_dialogue)
    file.write(f"\nTime taken: {elapsed_time:.2f} seconds")

print(f"=> Debate dialogue saved to {file_name}")
print(f"Time taken: {elapsed_time:.2f} seconds")

# Could be extended by another LLM analysing an long debate script and summarize it. Or be the judge on who won the debate and what the answer should be.  
# For a more sophisticated solution would include a expert panel that could vote on who would be the winner of the debate with different areas/principles to investigate. 

# TASK: 
    Discuss what is allowed and important to think about for the case of when a Chalmers employee wants to invite a professor with the purpose of recruitment.
    What and how much can Chalmers pay for such as flights, food, drinks?
    Moreover, what do Chalmers offer such as relocation, familiy jobs, etc.

__________Turn number 1__________
### Devil:
The university's governing document states, "Redan anställd fakultet ska premieras med möjlighet till ökad basfinansiering när man presterar excellent... Genomföra en rekryteringspilot med erbjudande om bra villkor till externa kandidater." Chalmers can exploit this by offering attractive financial incentives to recruit professors, potentially stretching the budget limits.


### Angel:
The university's commitment to providing increased funding to existing faculty for excellence does not imply unlimited budget flexibility for recruiting external candidates. As stated, the focus is on rewarding current staff. Thus, Chalmers must e